# GrEx3
For this assignment, I will be processing json files for perceptual mapping. This will include numeric perceptual mapping and text data for perceptual mapping. I will be exploring json files from customer service surveys for hotels.

# Part 1
Part 1 will be processing numeric perceptual mapping for the data. First I will import my packages

In [ ]:
import json
import os
import pandas as pd
from pandas.io.json import json_normalize
from html.parser import HTMLParser
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import nltk
from nltk.corpus import stopwords 
from string import punctuation
from collections import Counter

# Part 1 (a)

After Imporing my packages, I will write a loop to process the files in my directory.

In [ ]:
file_list = os.listdir('.') 
file_list

company_list=[]
for file in file_list:
    with open(file) as input_file:
        jsondat = json.load(input_file)
        reviews = jsondat['Reviews']
        hotel_info = jsondat['HotelInfo']
        reviews_df = json_normalize(reviews)
        hotels_df = json_normalize(hotel_info)
        reviews_df['HotelID'] = hotels_df.iloc[0]['HotelID']
        df = pd.DataFrame(reviews_df)
        company_list.append(df)
        
company_list

Each of the customer review files will have to be concattenated together for analysis.

In [ ]:
reviews_final = pd.concat(company_list)
reviews_final

Another list will have to be compiled in order to retrive a hotel's name and have it added to each record. This was performed this way because in some cases, a hotel's name was not available.

In [ ]:
f_list = os.listdir('.') 
f_list

hotel_list=[]

for file in f_list:
    with open(file) as input_file:
        jsondat2 = json.load(input_file)
        hotel_in = jsondat2['HotelInfo']
        hotel_df = json_normalize(hotel_in)
        hotel_list.append(hotel_df)
hotel_list

The hotel list was concattendated:

In [ ]:
hotel_final = pd.concat(hotel_list)
hotel_final

Then it was made into a dataframe:

In [ ]:
hotel_final = pd.DataFrame(hotel_final)
hotel_final

Beautiful Soup was used to take out the html tags and clean up the dataframe

In [ ]:
from bs4 import BeautifulSoup   #First install beautifulsoup4 using Canopy Package Manager
soup = BeautifulSoup(hotel_final['Address'], 'html.parser')
clean_address = soup.get_text().strip()    # get text and strip it of enclosing white space
hotel_final['Address'] = clean_address     # save the clean address to the deal_info
hotel_final

The hotel information was then normalized and merged with the reviews dataframe for analysis:

In [ ]:
hdf_final = json_normalize(hotel_final) #normalize the hotel dataframe
        
revs_df = reviews_final.merge(hotel_final) # merge the reviews dataframe with the hotel final dataframe

# Part 1 (b)
The number of reviews for each hotel in the dataframe is found

In [ ]:
revs_df.groupby('HotelID').Title.nunique()

# Part 1 (c)
The reported statistics describing the distribution of overall rating received by the hotels is found after checking the columns of the data frame

In [ ]:
revs_df.columns #Check the column headers
pd.to_numeric(revs_df['Ratings.Overall']) #Make sure the column is numeric

revs_df['Ratings.Overall'].describe() #find the statistics of the column

# Part 1 (d)
Save the dataframe by picking it:

In [ ]:
import pickle
revs_df.to_pickle("revs_df")

# Part 2
In this part, the data from reviews in the json data will be transformed in order to do perceptual mapping and finding hot words. I first tested the hotwords propcess in finding comments with one file before moving on to do the full analysis.

In [ ]:
with open('72572.json') as input_file:
    jsondat = json.load(input_file)

jsondat.keys()
jsondat['HotelInfo']
hotel_info = jsondat['HotelInfo']
hotel_info.keys()
hotel_info['Address']
reviews=jsondat['Reviews']
len(reviews)
reviews[0]
reviews[0].keys()
reviews[0]['Ratings']
reviews[0]['Ratings']['Service']
comments=reviews[0]['Content']
comments

# Part 2 (1-2)
Each hotel received a dictionary of comment content words and their frequencies and the counts of their occurences. A loop was written to process all json files at once instead of having to do each one individually.

In [ ]:
reviews_df_list = [] # to store the reviews DataFrame for each hotel
hotWords = {} 
file_list = os.listdir('.') 
file_list
for filename in file_list:
   with open(filename) as input_file:
    jsondat = json.load(input_file)
    jsondat.keys()
    hotel_info = jsondat['HotelInfo']
    hotel_info.keys()
    reviews = jsondat['Reviews']
    reviews_df_list.append(reviews)
    reviews = json_normalize(reviews)
    reviews
    reviews_df = pd.DataFrame(reviews)
    comments = reviews_df['Content'].str.cat(sep=' ')
    sentences = sent_tokenize(comments)
    words = word_tokenize(comments)
    cust_stop_words = set(stopwords.words('english')+list(punctuation))  
    filtered_words = [word for sent in sentences for word in word_tokenize(sent) if word.lower() not in cust_stop_words]
    filtered_words
    fw_freq = nltk.FreqDist(filtered_words).most_common()  
    fw_freq
    hotWords[hotel_info['HotelID']] = fw_freq #Add the key HotelID with the value fw_freq to the dict

# Part 2 (3)
The hotWords dict was written for a json file and was verified that it was written correctly:

In [ ]:
json = json.dumps(hotWords)
f = open("hotWords.json","w")
f.write(json)
f.close()

with open("hotWords.json", 'r') as fh:
    hotWords = json.load(fh)

# Part 2 (4)
The number of unique content words in each of the hotel's dicts were counted and printed:

In [ ]:
from itertools import chain
print(Counter(chain.from_iterable(e,keys() for e in hotWords)))

We also kept track of the most common hot words in each dictionary

In [ ]:
for hotelID in hotWords:
    print(hotelID,sorted(hotWords[hotelID])[:5])
from collections import Counter

for key in hotWords.keys():
    print(key,Counter(hotWords[key]).most_common(3))